In [261]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen, urlparse
import re
import time
import requests
import json

In [262]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [263]:
def engine(pages) : 
    url = url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=enddate&seed=2743718&page='
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    lst = []
    for i in range(1,pages) :
        r = requests.get(url+str(i),headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        lst.append(soup)
        i =+ 1   
    return lst

In [264]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

In [265]:
def to_string(container) : 
    whole_string = str(container).replace('&amp', '').replace('&quot;', '')
    return whole_string

In [266]:
def get_name(whole_string) : 
    if '"name"' in whole_string:
        project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
    else:
        project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]
    project_info_dict = scraper_prototype(project_info, 'project')
    df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()
    return df1

In [267]:
def get_creator(whole_string) : 
    if '"creator"' in whole_string:
        creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
    else:
        creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]
    creator_info_dict = scraper_prototype(creator_string, 'creator')
    creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
    df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()
    
    return df2

In [268]:
def get_location(whole_string) : 
    if '"location"' in whole_string:
        location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
    else:
        location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]
    location_info_dict = scraper_prototype(location_string, 'location')
    df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()
    return df3

In [269]:
def make_a_row(df1,df2,df3) : 
    df = pd.concat((df1, df2, df3), axis=1)
    return df

In [270]:
def row_flow(container) : 
    string = to_string(container)
    name = get_name(string)
    creator = get_creator(string)
    location = get_creator(string)
    #profile = get_profile(string)
    row = make_a_row(name,creator,location)
    return row

In [271]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [272]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
if r.status_code == 200 : 
    print('Reading the Website')
else : 
    print('Something went wrong')

Reading the Website


In [273]:
containers = soup.find_all('div', class_="js-react-proj-card")

In [274]:
row = row_flow(containers[0])
row0 = row_flow(containers[1])
row1 = row_flow(containers[2])
row2 = row_flow(containers[3])
row3 = row_flow(containers[4])
row4 = row_flow(containers[5])
row5 = row_flow(containers[6])
row6 = row_flow(containers[7])
row7 = row_flow(containers[8])
row8 = row_flow(containers[9])
row9 = row_flow(containers[10])
row10 = row_flow(containers[11])

In [286]:
df = pd.concat([row,row0,row1,row2,row3,row4,row6,row7,row8,row9,row10],ignore_index=True)

In [287]:
df = df.loc[:,~df.columns.duplicated()]

In [289]:
df.drop(['project_disable_communication','project_country_displayable_name',
         'project_currency_symbol','project_current_currency',
         'project_currency_trailing_code','project_staff_pick','project_backers_count',
         'project_static_usd_rate','project_converted_pledged_amount','project_fx_rate',
         'project_usd_exchange_rate','creator_is_registered','creator_is_email_verified',
         'creator_chosen_currency','creator_is_superbacker','creator_avatar','project_usd_pledged','project_is_starrable',
        'creator_small','creator_slug','project_currency'],axis=1,inplace=True)

In [317]:
df.set_index("creator_id", inplace = True)

In [291]:
lst =[]
my_dict ={}
for i in range(0,len(df)) :
    creator_id = str(df.loc[i]['creator_id'])
    html = requests.get('https://www.kickstarter.com/profile/'+creator_id+'/about', headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        #print(str(df.loc[i]['creator_id']))
        lst.append((a['href'], creator_id ))
        my_dict[creator_id]=(a['href'])

In [293]:
for el in lst : 
    if is_valid(el[0]) != True :
        lst.remove(el)
    if el[0].find('http') != -1 :
        lst2.append(el)
        

In [295]:
s = []
for el in  lst2: 
    try :
        r = requests.get(el[0])
        t = str(r.text)
        shopify = t.find('shopify')
        s.append((shopify,el))
    except requests.exceptions.SSLError:
        pass

In [313]:
last_lst = []
for el in s : 
    if el[0] < 0 :
        s.remove(el)
    else : 
        last_lst.append(el)

In [314]:
if len(s) == 0 : 
    print('No shopify user for now')
else : 
    print(len(s),' Shopify user detected')

1  Shopify user detected


In [315]:
last_lst

[(1020, ('http://midnight-cards.myshopify.com/', '42914364'))]

In [318]:
for el in last_lst : 
    infos = el[1]
    print(df.loc[infos[1]])

project_name                                         RED RISING Playing Cards
project_blurb               Officially-Licensed Red Rising Playing Cards! ...
project_goal                                                          10000.0
project_pledged                                                      125729.0
project_state                                                            live
project_slug                                         red-rising-playing-cards
project_country                                                            US
project_currency                                                          USD
project_deadline                                                   1646326722
project_state_changed_at                                           1643734723
project_created_at                                                 1638743533
project_launched_at                                                1643734722
creator_name                               Randy Butterfield (Mi

In [319]:
new_df = df.loc[infos[1]]

In [320]:
new_df['website'] = infos[0]

In [321]:
new_df

project_name                                         RED RISING Playing Cards
project_blurb               Officially-Licensed Red Rising Playing Cards! ...
project_goal                                                          10000.0
project_pledged                                                      125729.0
project_state                                                            live
project_slug                                         red-rising-playing-cards
project_country                                                            US
project_currency                                                          USD
project_deadline                                                   1646326722
project_state_changed_at                                           1643734723
project_created_at                                                 1638743533
project_launched_at                                                1643734722
creator_name                               Randy Butterfield (Mi

In [322]:
result = new_df.to_json(orient="split")

In [323]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False)